In [1]:
import numpy as np
import pandas as pd
from numpy import dtype
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
dtype={'tiprel_1mes': np.str, 'ult_fec_cli_1t':np.str, 'indrel_1mes':np.str}

parse_dates = ['fecha_alta', 'ult_fec_cli_1t']

iter_csv = pd.read_csv('../data/processed/train-processed.csv.gz', skipinitialspace = True, 
                       iterator=True, chunksize=50000, dtype=dtype, parse_dates=parse_dates)

data = pd.concat([chunk for chunk in iter_csv], ignore_index=True)

/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
# "removed corrupted data and redundant"
data.drop(['conyuemp', 'tipodom'], axis=1, inplace=True) # Contains just single diff value. All others same

# "converting dates to timestamps"
data.fecha_alta = data.fecha_alta.astype('int64')//1e9
data.fecha_alta[data.fecha_alta < 0] = 0

data.ult_fec_cli_1t = data.ult_fec_cli_1t.astype('int64')//1e9
data.ult_fec_cli_1t[data.ult_fec_cli_1t < 0] = 0

# "ind_empleado"
ind_empleado = pd.get_dummies(data.ind_empleado, prefix = 'ind_empleado')
data.drop('ind_empleado', axis=1, inplace=True)
data = pd.concat([data, ind_empleado], axis=1)

# "tiprel_1mes"
tiprel_1mes = pd.get_dummies(data.tiprel_1mes, prefix = 'tiprel_1mes')
data.drop('tiprel_1mes', axis=1, inplace=True)
data = pd.concat([data, tiprel_1mes], axis=1)

data.indrel = data.indrel.replace([99.], [0.])
data.renta = data.renta.replace([float('nan')], [0.])
data.indfall = data.indfall.replace(['S','N'], [1.,0.])
data.cod_prov = data.cod_prov.replace([float('nan')], [0.])
data.canal_entrada = data.canal_entrada.replace([None], ['nan'])
data.indrel_1mes = data.indrel_1mes.replace(['P', float('nan')], [5., 6.])
data.indrel_1mes = pd.to_numeric(data.indrel_1mes)

data.head()

/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/petr/Applications/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,indresi,indext,canal_entrada,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R
0,15889,ES,1.0,56.0,790214400,0.0,255.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,15890,ES,1.0,63.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,15891,ES,0.0,59.0,1438041600,0.0,246.0,0.0,1438732800,1.0,1.0,0.0,KAT,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,15892,ES,0.0,62.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,15893,ES,1.0,63.0,875836800,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


### coordinates feature
Load external dictionary of coordinates and replace code with coordinates in dataset

In [5]:
countries = pd.read_csv('../data/external/countries.csv', header=None)
countries.columns=['pais_residencia', 'lat', 'lon']

data = data.merge(countries, on='pais_residencia')
data.drop(['canal_entrada'], axis=1, inplace=True) #pais_residencia
data.sort_values(['ncodpers'], inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,lat,lon
0,15889,ES,1.0,56.0,790214400,0.0,255.0,1.0,0,1.0,1.0,0.0,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0,-4.0
1,15890,ES,1.0,63.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0,-4.0
2,15891,ES,0.0,59.0,1438041600,0.0,246.0,0.0,1438732800,1.0,1.0,0.0,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,-4.0
3,15892,ES,0.0,62.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0,-4.0
4,15893,ES,1.0,63.0,875836800,0.0,256.0,1.0,0,1.0,1.0,0.0,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,40.0,-4.0


In [7]:
test = pd.read_csv('../data/raw/test.csv.zip', usecols=['ncodpers'])
len(test[~test.ncodpers.isin(data.ncodpers)])

5

In [8]:
new_cols_order = ['ncodpers', 'sexo', 'age', 'lat', 'lon', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'ind_empleado_A', 'ind_empleado_B', 'ind_empleado_F', 'ind_empleado_N', 'ind_empleado_S', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes_A', 'tiprel_1mes_I', 'tiprel_1mes_P', 'tiprel_1mes_R', 'indresi', 'indext', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
data = data[new_cols_order]
data.head()

,ncodpers,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,1.0,56.0,40.0,-4.0,790214400,0.0,255.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,15890,1.0,63.0,40.0,-4.0,790214400,0.0,256.0,1.0,1.0,0.0,0.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,15891,0.0,59.0,40.0,-4.0,1438041600,0.0,246.0,0.0,0.0,0.0,0.0,1.0,0.0,1438732800,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,15892,0.0,62.0,40.0,-4.0,790214400,0.0,256.0,1.0,0.0,0.0,1.0,0.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,15893,1.0,63.0,40.0,-4.0,875836800,0.0,256.0,1.0,0.0,0.0,0.0,1.0,0.0,0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [9]:
data.to_csv('../data/processed/train-processed.csv.gz', index=False, compression='gzip')

In [10]:
clean_data_without_id = data.drop('ncodpers', axis=1)

x = clean_data_without_id.values #returns a numpy array
x_scaled = MinMaxScaler().fit_transform(x)

df = pd.DataFrame(x_scaled)

In [11]:
df.columns=list(clean_data_without_id.columns.values)
df.tail()

,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
949604,1.0,0.308642,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949605,0.0,0.172840,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.788462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949606,1.0,0.117284,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949607,0.0,0.253086,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
949608,1.0,0.234568,0.764151,0.355072,1.0,1.0,0.999744,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.403846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
normalized_data = pd.concat([data['ncodpers'], df], axis=1)
normalized_data.to_csv('../data/processed/train-normalized.csv.gz', index=False, compression='gzip')

In [ ]:
len(test[~test.ncodpers.isin(normalized_data.ncodpers)])